## Distribute Workload using ZeroMQ

In this section, we will be using ZeroMQ to distribute workload of generating temporal traces
among distributed workers. Doing so, gives us the ability to get a much higher throughput of
simulations.

In [1]:
import sys
import time
import random
from threading import Thread
import struct
import pickle

import zmq
from tqdm import tqdm
import pandas as pd

port = "5556"


context = zmq.Context()
socket = context.socket(zmq.DEALER)
socket.setsockopt(zmq.IDENTITY, b'master')
socket_addr = "tcp://127.0.0.1:%s" % port
socket.bind(socket_addr)

poller = zmq.Poller()
poller.register(socket, zmq.POLLIN)

In [2]:
# Test API function before running worker
from examples.sim_trace import generate_trace_api

data = {
    "arrival_rate": 0.9, 
    "warm_service_rate": 1/2.016, 
    "cold_service_rate": 1/2.163,
    "expiration_threshold": 600, 
    "max_time": 10000,
}

generate_trace_api(data)

{'reqs_cold': 22,
 'reqs_total': 9120,
 'reqs_warm': 9098,
 'prob_cold': 0.002412280701754386,
 'reqs_reject': 0,
 'prob_reject': 0.0,
 'lifespan_avg': 4439.236638325309,
 'inst_count_avg': 7.950966171119343,
 'inst_running_count_avg': 1.8464545888184838,
 'inst_idle_count_avg': 6.104511582300859,
 'arrival_rate': 0.9,
 'warm_service_rate': 0.49603174603174605,
 'cold_service_rate': 0.46232085067036527,
 'expiration_threshold': 600,
 'max_time': 10000}

In [3]:
total_sims = 100

# generate request that will be sent out to the worker
request = pickle.dumps(data)
def sender(num):
    # send the tasks
    for _ in range(num):
        socket.send(request)

st = Thread(target=sender, args=(total_sims,))
st.start()

In [4]:
pbar = tqdm(total=int(total_sims))

received_sims = 0
results = []
# receive the results
while received_sims < total_sims:
    socks = dict(poller.poll(timeout=5000))
    if socks == {}:
        print("Timeout!")
        break

    if socket in socks and socks[socket] == zmq.POLLIN:
        # print("Message from socket: %s" % struct.unpack("d", socket.recv()))
        results.append(pickle.loads(socket.recv()))
        received_sims += 1
        pbar.update(1)

pbar.close()

100%|██████████| 100/100 [00:09<00:00, 11.07it/s]


In [5]:
res_df = pd.DataFrame(data=results)
print(f"The probability of cold start is: {res_df['prob_cold'].mean():.8f}")

res_df.head()

The probability of cold start is: 0.00208998


,reqs_cold,reqs_total,reqs_warm,prob_cold,reqs_reject,prob_reject,lifespan_avg,inst_count_avg,inst_running_count_avg,inst_idle_count_avg,arrival_rate,warm_service_rate,cold_service_rate,expiration_threshold,max_time
0,16,8981,8965,0.001782,0,0.0,3971.685914,7.573530,1.802218,5.771312,0.9,0.496032,0.462321,600,10000
1,18,8902,8884,0.002022,0,0.0,4720.146055,7.634842,1.787555,5.847287,0.9,0.496032,0.462321,600,10000
2,26,8972,8946,0.002898,0,0.0,3575.223710,8.130272,1.788940,6.341331,0.9,0.496032,0.462321,600,10000
3,18,9054,9036,0.001988,0,0.0,4811.299354,7.534811,1.859861,5.674950,0.9,0.496032,0.462321,600,10000
4,25,8957,8932,0.002791,0,0.0,3574.218239,7.887112,1.824521,6.062591,0.9,0.496032,0.462321,600,10000


In [7]:
res_df.mean().to_dict()

{'reqs_cold': 18.82,
 'reqs_total': 9005.23,
 'reqs_warm': 8986.41,
 'prob_cold': 0.0020899819234118677,
 'reqs_reject': 0.0,
 'prob_reject': 0.0,
 'lifespan_avg': 4574.026199719348,
 'inst_count_avg': 7.683450441510985,
 'inst_running_count_avg': 1.8181519008460987,
 'inst_idle_count_avg': 5.86529854066488,
 'arrival_rate': 0.9000000000000008,
 'warm_service_rate': 0.49603174603174677,
 'cold_service_rate': 0.4623208506703657,
 'expiration_threshold': 600.0,
 'max_time': 10000.0}